<a href="https://colab.research.google.com/github/QaziSaim/CASE-STUDIES/blob/main/Advance_Transformer_Designing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %%capture
# !pip install --upgrade tensorflow

In [ ]:
import tensorflow as tf
tf.__version__

'2.19.0'

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from datasets import load_dataset
import numpy as np

### Loading Dataset

In [ ]:
dataset = load_dataset('wikitext','wikitext-2-raw-v1')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

wikitext-2-raw-v1/test-00000-of-00001.pa(…):   0%|          | 0.00/733k [00:00<?, ?B/s]

wikitext-2-raw-v1/train-00000-of-00001.p(…):   0%|          | 0.00/6.36M [00:00<?, ?B/s]

wikitext-2-raw-v1/validation-00000-of-00(…):   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
text = " ".join(dataset['train']['text'])
print('Total characters:',len(text))

Total characters: 10929707


In [ ]:
# dataset = load_dataset('wikitext','wikitext-2-raw-v1')
test_text = " ".join(dataset['test']['text'])
print('Total characters:',len(test_text))

Total characters: 1289979


In [ ]:
validate_text = " ".join(dataset['validation']['text'])
print('Total characters:',len(validate_text))

Total characters: 1145909


### Tokenization

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

In [ ]:
tokenizer = Tokenizer(num_words=10000, oov_token='<oov>')
tokenizer.fit_on_texts(dataset['train']['text'])
vocab_size = len(tokenizer.word_index) + 1
print('Vocab Size ',vocab_size)
tokens = tokenizer.texts_to_sequences([text])[0]

Vocab Size  66008


In [ ]:
test_tokenizer = Tokenizer(num_words=10000, oov_token='<oov>')
test_tokenizer.fit_on_texts(dataset['test']['text'])
test_vocab_size = len(test_tokenizer.word_index) + 1
print('Vocab Size ',test_vocab_size)
test_tokens = test_tokenizer.texts_to_sequences([test_text])[0]

Vocab Size  18714


In [ ]:
validate_tokenizer = Tokenizer(num_words=10000, oov_token='<oov>')
validate_tokenizer.fit_on_texts(dataset['validation']['text'])
validate_vocab_size = len(validate_tokenizer.word_index) + 1
print('Vocab Size ',validate_vocab_size)
validate_tokens = validate_tokenizer.texts_to_sequences([validate_text])[0]

Vocab Size  17489


In [ ]:
seq_length = 10
X_train = []
y_train = []

for i in range(seq_length, len(tokens)):
    X_train.append(tokens[i-seq_length:i])
    y_train.append(tokens[i])

X_train = np.array(X_train)
y_train = np.array(y_train)
print("Training samples:", X_train.shape)


Training samples: (1760497, 10)


In [ ]:
tokens[:10],validate_tokens[:10],test_tokens[:10],len(tokens),len(validate_tokens),len(test_tokens)

([3800, 3862, 868, 1, 77, 3800, 81, 1, 3862, 769],
 [1638, 825, 1638, 825, 164, 10, 2, 1096, 1752, 55],
 [1361, 1129, 1361, 1129, 19, 24, 830, 78, 756, 4],
 1760507,
 184734,
 207607)

In [ ]:
seq_length = 10
X_val = []
y_val = []

for i in range(seq_length, len(validate_tokens)):
    X_val.append(validate_tokens[i-seq_length:i])
    y_val.append(validate_tokens[i])

X_val = np.array(X_val)
y_val = np.array(y_val)
print("Validating samples:", X_val.shape)


Validating samples: (184724, 10)


In [ ]:
seq_length = 10
X_test = []
y_test = []

for i in range(seq_length, len(test_tokens)):
    X_test.append(test_tokens[i-seq_length:i])
    y_test.append(test_tokens[i])

X_test = np.array(X_test)
y_test = np.array(y_test)
print("Test samples:", X_test.shape)


Test samples: (207597, 10)


In [ ]:
X_train.shape

(1760497, 10)

In [ ]:
import tensorflow as tf

In [ ]:
def scaled_dot_product_attention(q, k, v, mask=None):  # Added mask param
    def valid_computation():
        matmul = tf.matmul(q, k, transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_logits = matmul / tf.math.sqrt(dk)

        # Apply causal mask if provided (for decoder-only)
        if mask is not None:
            scaled_logits += (mask * -1e9)  # Mask future positions with large negative value

        weights = tf.nn.softmax(scaled_logits, axis=-1)
        output = tf.matmul(weights, v)
        return output

    def empty_computation():
        return tf.zeros_like(v)

    batch_size_q = tf.shape(q)[0]
    batch_size_k = tf.shape(k)[0]
    batch_size_v = tf.shape(v)[0]

    return tf.cond(
        tf.logical_or(tf.equal(batch_size_q, 0), tf.logical_or(tf.equal(batch_size_k, 0), tf.equal(batch_size_v, 0))),
        empty_computation,
        valid_computation
    )

In [ ]:
class MultiHeadAttention(layers.Layer):
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0
        self.num_heads = num_heads
        self.depth = d_model // num_heads
        self.wq = layers.Dense(d_model)
        self.wk = layers.Dense(d_model)
        self.wv = layers.Dense(d_model)
        self.dense = layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, [0, 2, 1, 3])

    def call(self, q, k, v, mask=None):  # Added mask param
        batch_size = tf.shape(q)[0]
        q = self.split_heads(self.wq(q), batch_size)
        k = self.split_heads(self.wk(k), batch_size)
        v = self.split_heads(self.wv(v), batch_size)
        attn = scaled_dot_product_attention(q, k, v, mask=mask)  # Pass mask
        attn = tf.transpose(attn, [0, 2, 1, 3])
        concat = tf.reshape(attn, (batch_size, -1, self.num_heads * self.depth))
        return self.dense(concat)

In [ ]:
class DecoderLayer(layers.Layer):  # Renamed for clarity (optional)
    def __init__(self, d_model, num_heads, dff, rate=0.01):
        super().__init__()
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = tf.keras.Sequential([
            layers.Dense(dff, activation='relu'),
            layers.Dense(d_model)
        ])
        self.ln1 = layers.LayerNormalization(epsilon=1e-6)
        self.ln2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, x, training, mask=None):  # Added mask param
        attn = self.mha(x, x, x, mask=mask)  # Pass mask to attention
        out1 = self.ln1(x + self.dropout1(attn, training=training))
        ffn_out = self.ffn(out1)
        out2 = self.ln2(out1 + self.dropout2(ffn_out, training=training))
        return out2

In [ ]:
class PositionalEncoding(layers.Layer):
    def __init__(self, seq_length, d_model):
        super().__init__()
        self.d_model = d_model
        self.seq_length = seq_length
        self.pos_encoding = self.positional_encoding(seq_length, d_model)

    def get_angles(self, pos, i, d_model):
        angles = pos / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return angles

    def positional_encoding(self, seq_length, d_model):
        angle_rads = self.get_angles(
            pos=tf.range(seq_length, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])
        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, x):
        return x + self.pos_encoding[:, :tf.shape(x)[1], :]

In [ ]:
class Transformer(tf.keras.Model):
    def __init__(self, vocab_size, num_layers, d_model, num_heads, dff, seq_length, rate=0.1):
        super().__init__()
        self.embedding = layers.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(seq_length, d_model)
        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]  # Renamed list
        self.dropout = layers.Dropout(rate=rate)
        self.final = layers.Dense(vocab_size)

    def create_causal_mask(self, seq_len):
        # Create upper-triangle mask for causal attention
        mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)  # Lower triangle = 1 (attend), upper = 0 (mask)
        return mask[tf.newaxis, tf.newaxis, :, :]  # Broadcast for batch/heads
    def call(self, x, training=False):
      seq_len = tf.shape(x)[1]
      mask = self.create_causal_mask(seq_len)  # Always create mask (avoids tf.cond mismatch)

      x = self.embedding(x)
      x *= tf.math.sqrt(tf.cast(self.embedding.output_dim, tf.float32))
      x = self.pos_encoding(x)
      x = self.dropout(x, training=training)
      for layer in self.dec_layers:
          x = layer(x, training=training, mask=mask)
      x = self.final(x)
      return x[:, -1, :]

In [ ]:
# Learning rate schedule
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
config = {
    'vocab_size': vocab_size,
    'num_layers': 2,
    'd_model': 128,
    'num_heads': 4,
    'dff': 512,
    'seq_length': 50,
    'dropout_rate': 0.1,
    'batch_size': 64,
    'epochs': 50
}

In [ ]:
model = Transformer(
    vocab_size=config['vocab_size'],
    num_layers=config['num_layers'],
    d_model=config['d_model'],
    num_heads=config['num_heads'],
    dff=config['dff'],
    seq_length=config['seq_length'],
    rate=config['dropout_rate']
)
# Rest (compile, fit) unchanged

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [ ]:
import tensorflow as tf

def label_smoothed_sparse_categorical_crossentropy(label_smoothing=0.1):
    def loss(y_true, y_pred):
        # One-hot encode sparse y_true
        num_classes = tf.shape(y_pred)[-1]
        y_true_one_hot = tf.one_hot(tf.cast(y_true, tf.int32), depth=num_classes)

        # Apply label smoothing: (1 - smoothing) * one_hot + (smoothing / num_classes)
        smoothed_labels = y_true_one_hot * (1.0 - label_smoothing) + (label_smoothing / tf.cast(num_classes, tf.float32))

        # Compute crossentropy with smoothed labels
        return tf.keras.losses.categorical_crossentropy(
            smoothed_labels, y_pred, from_logits=True
        )
    return loss

In [ ]:
learning_rate = 3e-4
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
smoothed_loss = label_smoothed_sparse_categorical_crossentropy(label_smoothing=0.1)
model.compile(optimizer=optimizer,
              loss=smoothed_loss,
              metrics=['accuracy'])

model.summary()

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ positional_encoding             │ ?                      │   0 (unbuilt) │
│ (PositionalEncoding)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_layer (DecoderLayer)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_layer_1 (DecoderLayer)  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor="val_loss",
    patience=3,
    restore_best_weights=True,
)

In [ ]:
# ## for demo only
# from sklearn.model_selection import train_test_split
# X_main,X_test,y_main,y_test = train_test_split(X_train,y_train,test_size=0.2,random_state=42)
# X_train,X_val,y_train,y_val = train_test_split(X_main,y_main,test_size=0.25,random_state=42)

In [ ]:
print('shapes')
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)
print(X_val.shape,y_val.shape)

shapes
(1760497, 10) (1760497,)
(207597, 10) (207597,)
(184724, 10) (184724,)


In [ ]:
# model.fit(input_sequences, output_words, batch_size=64, epochs=5)
history = model.fit(X_train,y_train,batch_size=64,epochs=50,validation_data=(X_val,y_val),callbacks=[es])

Epoch 1/50
27508/27508 ━━━━━━━━━━━━━━━━━━━━ 251s 8ms/step - accuracy: 0.1337 - loss: 7.2354 - val_accuracy: 0.0794 - val_loss: 7.8391
Epoch 2/50
27508/27508 ━━━━━━━━━━━━━━━━━━━━ 208s 8ms/step - accuracy: 0.1652 - loss: 6.8993 - val_accuracy: 0.0750 - val_loss: 8.1354
Epoch 3/50
27508/27508 ━━━━━━━━━━━━━━━━━━━━ 223s 8ms/step - accuracy: 0.1701 - loss: 6.9736 - val_accuracy: 0.0741 - val_loss: 8.1172
Epoch 4/50
27508/27508 ━━━━━━━━━━━━━━━━━━━━ 214s 8ms/step - accuracy: 0.1745 - loss: 6.9493 - val_accuracy: 0.0733 - val_loss: 8.1812


In [ ]:
model.summary()

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 10, 128)        │     8,449,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ positional_encoding             │ ?                      │             0 │
│ (PositionalEncoding)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_layer (DecoderLayer)    │ ?                      │       198,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_layer_1 (DecoderLayer)  │ ?                      │       198,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 10, 66008)      │     8,515,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52,081,802 (198.68 MB)

 Trainable params: 17,360,600 (66.23 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 34,721,202 (132.45 MB)

In [ ]:
loss, accuracy = model.evaluate(X_test,y_test)
print(f'Test Accuracy {accuracy:.4f}')

6488/6488 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - accuracy: 0.0745 - loss: 7.8688
Test Accuracy 0.0768


In [ ]:
print(loss)

7.832433700561523


In [ ]:
def generate_text(seed_text, next_words=30, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=seq_length, padding='pre')
        predictions = model.predict(token_list, verbose=0)[0]
        predictions = predictions / temperature  # Add temperature for diversity
        predicted = np.random.choice(len(predictions), p=tf.nn.softmax(predictions).numpy())  # Sample for variety

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        if output_word == "":  # Fallback for OOV
            output_word = "<oov>"
        seed_text += " " + output_word
    return seed_text

print(generate_text("what do you know about computers", next_words=30, temperature=0.8))

what do you know about computers the audience <oov> of the eva murray <oov> wrote a new son and alabama match were <oov> with the village crew in <oov> 's <oov> of the george nation the


In [ ]:
import math
loss, _ = model.evaluate(X_test, y_test)
perplexity = math.exp(loss)
print(f"Perplexity: {perplexity:.2f}")


6488/6488 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.0745 - loss: 7.8688
Perplexity: 2521.06


In [ ]:
from nltk.translate.bleu_score import sentence_bleu

reference = [['this', 'is', 'a', 'test']]
candidate = ['this', 'is', 'test']
print("BLEU:", sentence_bleu(reference, candidate))


BLEU: 8.987727354491445e-155


/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [ ]:
# from bert_score import score
# P, R, F1 = score(["generated sentence"], ["reference sentence"], lang="en", verbose=True)
# print("BERTScore F1:", F1.mean().item())


In [ ]:
print(generate_text("The future of AI", next_words=50))


The future of AI reviewers grew off on each required the league and 200 filmmakers william further entirely bowl in sankalia on best reviews contamination from shakes off on april best stories she feed team testing 's 1973 nonplussed era asia in <oov> on military defense 's work of their missions as they 1998
